In [11]:
import os
from pypdf import PdfReader

import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display, Audio

import gradio as gr
import boto3

In [12]:
genai.configure(api_key = os.getenv("GOOGLE_API_KEY"))
polly = boto3.client('polly',region_name='us-east-1',aws_access_key_id='YOUR_ACCESS_KEY',aws_secret_access_key='YOUR_SECRET_KEY')

In [13]:
#pdf_path = "2311.14141.pdf"
def pdf_reader(pdf_path):
    reader = PdfReader(pdf_path) 
    text=" "
    for i in range(len(reader.pages)):
        page = reader.pages[i] 
        text += page.extract_text()
    return text

In [14]:
parsed_text = pdf_reader(pdf_path)
parsed_text
len(parsed_text)

55837

In [15]:
prompt = """YOUR_PROMPT """
def generate_gemini_content(transcript_text,prompt):
    model=genai.GenerativeModel("gemini-pro")
    response=model.generate_content(prompt+transcript_text)
    return response.text

In [16]:
final_text = generate_gemini_content(parsed_text,prompt)

In [22]:
response = polly.synthesize_speech(Text=final_text, OutputFormat="mp3",
                                       VoiceId="Joanna")
if "AudioStream" in response:
   with response["AudioStream"] as stream:
      output_file = "speech.mp3"
      try:
         # Open a file for writing the output as a binary stream
            with open(output_file, "wb") as file:
               file.write(stream.read())
      except IOError as error:
         # Could not write to file, exit gracefully
         print(error)
else:
   # The response didn't contain audio data, exit gracefully
   print("Could not stream audio")

In [28]:
def main(path_pdf):
   prompt = """YOUR_PROMPT """
   parsed_text = pdf_reader(path_pdf)
   final_text = generate_gemini_content(parsed_text,prompt)
   response = polly.synthesize_speech(Text=final_text, OutputFormat="mp3",
                                       VoiceId="Joanna")
   if "AudioStream" in response:
      with response["AudioStream"] as stream:
         output_file = "speech.mp3"
         try:
            # Open a file for writing the output as a binary stream
               with open(output_file, "wb") as file:
                  file.write(stream.read())
         except IOError as error:
            # Could not write to file, exit gracefully
            print(error)
   else:
      # The response didn't contain audio data, exit gracefully
      print("Could not stream audio")
   return final_text, output_file

In [8]:
pdf_path = "2311.14141.pdf"
main(pdf_path)

('**Summary**\n\nThis paper presents a novel turn-based encoding algorithm for predicting the structure of smaller protein sequences using the HP (Hydrophobic and Polar bead) model. The algorithm is designed to be run on a gate-based quantum computer and utilizes variational algorithms to optimize solutions. The main contributions of the paper are:\n\n* Development of a turn-based encoding formulation that allows for increased degrees of freedom in protein folding simulations.\n* Implementation of the formulation on a quantum computer using VQE and CVaR algorithms to estimate energy values.\n* Comparison of results with classical optimization algorithms and IBM Quantum simulators and hardwares.\n* Demonstration of hydrophobic collapse, a key phenomenon in protein folding, in both quantum and classical simulations.\n\n**Methodology**\n\nThe turn-based encoding formulation is used to represent the problem as a QUBO (Quadratic Unconstrained Binary Optimization) problem. This formulation i

In [23]:
from IPython.display import Audio
Audio(output_file, autoplay=True)

In [29]:
input_pdf_path = gr.File(label="Upload PDF")
output_summary = gr.components.Textbox(label="Summary"), gr.Audio()

interface = gr.Interface(
    fn=main,
    inputs=input_pdf_path,
    outputs=["text", "audio"],
    title="PDF Summarizer",
    description="Provide PDF file path to get the summary.",
).launch(share=True)

Running on local URL:  http://127.0.0.1:7864

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on public URL: https://5668b88518d4d6af62.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
